# Land Cover Classification of Satellite Images

**Aidan O'Keefe**

IMAGE GOES HERE

## Overview

A deep learning (neural network) land cover classification project using satellite images (remote sensing).